# G3 Generador de Animaciones

Generador que crea un video con rectángulos de colores y movimiento aleatoria.

## Importar Librerías

- `os`: módulo estándar para operaciones del sistema (rutas, crear carpetas). En este código se usa os.makedirs y os.path.join para preparar la carpeta y las rutas de salida.
- `random`: Módulo estándar para generación de números aleatorios; aquí se usa para posiciones, velocidades y colores aleatorios de los rectángulos.
- `numpy`: librería para arrays y operaciones numéricas; se utiliza para crear los frames como arrays uint8 con forma (height, width, 3) y pintar los rectángulos eficientemente.
- `imageio.v2 / imageio`: librería para leer/escribir imágenes y videos; se usa imageio.get_writer (con códec libx264) para escribir MP4 y imageio.mimsave como alternativa para guardar GIFs.

In [1]:
import os 
import random 
import numpy as np 
import imageio.v2 as imageio

## Configuración de Parámetros de la Animación

Definimos los parámetros principales que controlan la resolución, duración y características de la animación:

- **width, height**: Dimensiones del frame en píxeles (640x360 = resolución SD)
- **fps**: Fotogramas por segundo (30 fps es estándar para video suave)
- **duration_seconds**: Duración total de la animación en segundos
- **num_frames**: Total de fotogramas = fps × duración
- **num_rectangles**: Cantidad de rectángulos animados
- **rect_w, rect_h**: Dimensiones de cada rectángulo (ancho × alto)

In [2]:
# Configuración de resolución y velocidad
width, height = 640, 360 
fps = 30 
duration_seconds = 5 
num_frames = fps * duration_seconds 

# Configuración de los rectángulos
num_rectangles = 8 
rect_w, rect_h = 60, 40 

print(f"Resolución: {width}x{height}")
print(f"FPS: {fps}")
print(f"Duración: {duration_seconds} segundos")
print(f"Total de frames: {num_frames}")
print(f"Rectángulos: {num_rectangles}")

Resolución: 640x360
FPS: 30
Duración: 5 segundos
Total de frames: 150
Rectángulos: 8


## Inicializar Rectángulos

Creamos una lista de rectángulos, cada uno con:
- **x, y**: Posición inicial aleatoria (dentro de los límites del frame)
- **vx, vy**: Velocidades en X e Y (velocidad en píxeles/frame)
- **color**: Color RGB aleatorio (valores entre 80-255 para evitar colores muy oscuros)

Cada rectángulo se representa como un diccionario para facilitar el acceso a sus propiedades.

In [3]:
rectangles = [] 

for i in range(num_rectangles): 
    rect = { 
        "x": random.randint(0, width - rect_w),   # Posición X inicial
        "y": random.randint(0, height - rect_h),  # Posición Y inicial
        "vx": random.choice([-3, -2, -1, 1, 2, 3]),  # Velocidad X (píxeles/frame)
        "vy": random.choice([-3, -2, -1, 1, 2, 3]),  # Velocidad Y (píxeles/frame)
        "color": (                                  # Color RGB aleatorio
            random.randint(80, 255), 
            random.randint(80, 255), 
            random.randint(80, 255), 
        ), 
    } 
    rectangles.append(rect) 

print(f"Rectángulos creados: {len(rectangles)}")

# Mostrar información de los primeros rectángulos
for i, rect in enumerate(rectangles[:3]):
    print(f"  Rectángulo {i}: pos=({rect['x']}, {rect['y']}), vel=({rect['vx']}, {rect['vy']}), color={rect['color']}")

Rectángulos creados: 8
  Rectángulo 0: pos=(506, 243), vel=(-1, 1), color=(197, 105, 215)
  Rectángulo 1: pos=(389, 50), vel=(-3, -2), color=(203, 139, 150)
  Rectángulo 2: pos=(11, 12), vel=(-3, -3), color=(165, 176, 214)


## Generar Frames de la Animacion

Para cada frame:
1. Creamos una matriz numpy de pixeles (altura x ancho x 3 canales RGB) inicializada en negro (0,0,0)
2. Actualizamos la posicion de cada rectangulo sumando su velocidad
3. Calculamos las esquinas y recortamos (clip) a los limites del frame para evitar indices fuera de rango
4. Dibujamos el rectangulo en el frame asignando su color a la region correspondiente
5. Almacenamos el frame en una lista

**Nota**: Si un rectangulo sale parcialmente del area visible, se recorta; si sale por completo, no se dibuja.

In [ ]:
frames = []

for frame_idx in range(num_frames):

    frame = np.zeros((height, width, 3), dtype=np.uint8)

    for rect in rectangles:
        # Mover el rectangulo
        rect["x"] += rect["vx"]
        rect["y"] += rect["vy"]

        # Calcular esquinas del rectangulo
        x0, y0 = rect["x"], rect["y"]
        x1, y1 = x0 + rect_w, y0 + rect_h

        # Recortar a limites del frame
        x0_clipped = max(0, int(x0))
        x1_clipped = min(width, int(x1))
        y0_clipped = max(0, int(y0))
        y1_clipped = min(height, int(y1))

        # Dibujar solo si hay area visible
        if x0_clipped < x1_clipped and y0_clipped < y1_clipped:
            frame[y0_clipped:y1_clipped, x0_clipped:x1_clipped] = rect["color"]

    frames.append(frame)

print(f"Frames generados: {len(frames)}")

Frames generados: 150


## Guardado la Animación

Intentamos guardar la animación como video MP4. Si hay un error, la guardamos como GIF.

In [5]:
# Crear directorio de salida si no existe
output_dir = "output" 
os.makedirs(output_dir, exist_ok=True) 

# Definir rutas de salida
output_mp4 = os.path.join(output_dir, "G3-rectangulos-animacion.mp4") 
output_gif = os.path.join(output_dir, "G3-rectangulos-animacion.gif") 

print(f"Guardando animación en: {output_dir}/")

try:
    # Intentar guardar como MP4
    with imageio.get_writer(output_mp4, fps=fps, codec="libx264") as writer:
        for frame_idx, frame in enumerate(frames):
            writer.append_data(frame)
            if (frame_idx + 1) % 30 == 0:
                print(f"  {frame_idx + 1}/{len(frames)} frames escritos...")
    print(f"✓ Video MP4 guardado en: {output_mp4}")
except Exception as exc:
    # Si falla, guardar como GIF
    print(f"⚠ MP4 falló ({exc}), guardando como GIF...")
    imageio.mimsave(output_gif, frames, fps=fps)
    print(f"✓ Video GIF guardado en: {output_gif}")

Guardando animación en: output/


IMAGEIO FFMPEG_WRITER WARNING: input image is not divisible by macro_block_size=16, resizing from (640, 360) to (640, 368) to ensure video compatibility with most codecs and players. To prevent resizing, make your input image divisible by the macro_block_size or set the macro_block_size to 1 (risking incompatibility).


  30/150 frames escritos...
  60/150 frames escritos...
  90/150 frames escritos...
  120/150 frames escritos...
  150/150 frames escritos...
✓ Video MP4 guardado en: output\G3-rectangulos-animacion.mp4
